In [13]:
import mbuild as mb
import foyer
import hoomd
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

esp_ff = foyer.Forcefield(forcefield_files="/home/madilyn/Projects/repos/forcefields/CPDT.xml")

# We have to add the underscore to the names manually if we are using foyer XML files without SMARTS definitions
CPDT = mb.load("/home/madilyn/Projects/repos/forcefields/CPDT_typed.mol2")

for p in CPDT.particles():
    p.name = f"_{p.name}"
    
box = mb.fill_box(compound=CPDT, n_compounds=2, box=[10,10,10])
box_pmd = esp_ff.apply(box)

snapshot, forcefield, refs = create_hoomd_forcefield(box_pmd, auto_scale=True, r_cut=2.5)

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


/home/madilyn/miniconda3/envs/esp/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C2
  warnings.warn(
/home/madilyn/miniconda3/envs/esp/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H6
  warnings.warn(
/home/madilyn/miniconda3/envs/esp/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C1
  warnings.warn(
/home/madilyn/miniconda3/envs/esp/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H8
  warnings.warn(
/home/madilyn/miniconda3/envs/esp/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C4
  warnings.warn(
/home/madilyn/miniconda3/envs/esp/lib/python3.10/site-packages/fo

In [14]:
#setting cpu and simulation 
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snapshot)

#setting the integrator
kt = 1.2
free_particle = hoomd.filter.Tags(tags=[0,1])  #letting hoomd know which particles to update the positions of
integrator = hoomd.md.Integrator(dt = 0.01)
nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), kT=1.2, tau=0.5) #what is tau?
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=free_particle, kT=kt)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_CPDT.gsd",  #name the output file
    mode="wb"
)
sim.operations.writers.append(gsd_writer)

In [15]:
sim.run(1e4)

*Warning*: charge.pppm: RMS error of 0.130631 is probably too high! 0.130627 0.130631
**ERROR**: Particle with unique tag 33 is no longer in the simulation box.

Cartesian coordinates: 
x: 13.2444 y: 20.1421 z: 13.7768
Fractional coordinates: 
f.x: 0.971976 f.y: 1.21778 f.z: 0.990951
Local box lo: (-14.0308, -14.0308, -14.0308)
          hi: (14.0308, 14.0308, 14.0308)


RuntimeError: Error computing cell list